In [1]:
import mmdet
print(mmdet.__version__)

# 测试导入
from mmdet.apis import init_detector, inference_detector
print("✅ MMDetection安装成功!")

2.25.1
✅ MMDetection安装成功!


In [6]:
import os
import json
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
from pycocotools import mask as maskUtils
from datetime import datetime
from collections import defaultdict

class VOCToCOCOConverter:
    def __init__(self, voc_root, output_dir):
        self.voc_root = voc_root
        self.output_dir = output_dir
        self.categories = self._get_voc_categories()
        
    def _get_voc_categories(self):
        """VOC 2012的20个类别"""
        categories = [
            'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
            'bus', 'car', 'cat', 'chair', 'cow',
            'diningtable', 'dog', 'horse', 'motorbike', 'person',
            'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
        ]
        return {cat: i+1 for i, cat in enumerate(categories)}
    
    def _parse_xml(self, xml_file):
        """解析VOC XML标注文件"""
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        objects_info = []
        size = root.find('size')
        img_width = int(size.find('width').text)
        img_height = int(size.find('height').text)
        
        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in self.categories:
                continue
                
            bbox_elem = obj.find('bndbox')
            xmin = float(bbox_elem.find('xmin').text)
            ymin = float(bbox_elem.find('ymin').text)
            xmax = float(bbox_elem.find('xmax').text)
            ymax = float(bbox_elem.find('ymax').text)
            
            # COCO格式: [x, y, width, height]
            bbox = [xmin, ymin, xmax - xmin, ymax - ymin]
            
            objects_info.append({
                'category_name': name,
                'category_id': self.categories[name],
                'bbox': bbox,
                'iscrowd': 0
            })
            
        return objects_info, img_width, img_height
    
    def _extract_instance_masks(self, seg_file, objects_info, img_width, img_height):
        """从VOC分割图像中提取每个实例的mask"""
        if not os.path.exists(seg_file):
            return []
            
        # 读取分割图像
        seg_img = np.array(Image.open(seg_file))
        
        # 获取所有唯一的像素值（除了背景0和边界255）
        unique_values = np.unique(seg_img)
        unique_values = unique_values[(unique_values != 0) & (unique_values != 255)]
        
        annotations = []
        
        # 为每个检测到的对象匹配分割mask
        for obj_info in objects_info:
            # 在分割图像中寻找与边界框重叠最大的分割区域
            x, y, w, h = obj_info['bbox']
            x, y, w, h = int(x), int(y), int(w), int(h)
            
            # 获取边界框区域内的分割值
            bbox_region = seg_img[y:y+h, x:x+w]
            bbox_unique = np.unique(bbox_region)
            bbox_unique = bbox_unique[(bbox_unique != 0) & (bbox_unique != 255)]
            
            if len(bbox_unique) == 0:
                # 如果没有找到分割区域，使用边界框创建矩形mask
                mask = np.zeros((img_height, img_width), dtype=np.uint8)
                mask[y:y+h, x:x+w] = 1
            else:
                # 选择在边界框内占比最大的分割值
                best_value = None
                max_overlap = 0
                
                for val in bbox_unique:
                    if val in unique_values:
                        val_mask = (seg_img == val).astype(np.uint8)
                        overlap = np.sum(val_mask[y:y+h, x:x+w])
                        if overlap > max_overlap:
                            max_overlap = overlap
                            best_value = val
                
                if best_value is not None:
                    mask = (seg_img == best_value).astype(np.uint8)
                    # 从unique_values中移除已使用的值，避免重复使用
                    unique_values = unique_values[unique_values != best_value]
                else:
                    # 备选方案：使用边界框
                    mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    mask[y:y+h, x:x+w] = 1
            
            # 计算面积
            area = float(np.sum(mask))
            
            if area > 0:
                # 转换为RLE格式
                rle = maskUtils.encode(np.asfortranarray(mask))
                rle['counts'] = rle['counts'].decode('utf-8')
                
                # 从mask重新计算更精确的边界框
                coords = np.where(mask)
                if len(coords[0]) > 0:
                    y_min, y_max = coords[0].min(), coords[0].max()
                    x_min, x_max = coords[1].min(), coords[1].max()
                    precise_bbox = [float(x_min), float(y_min), 
                                  float(x_max - x_min + 1), float(y_max - y_min + 1)]
                else:
                    precise_bbox = obj_info['bbox']
                
                annotations.append({
                    'category_id': obj_info['category_id'],
                    'bbox': precise_bbox,
                    'area': area,
                    'segmentation': rle,
                    'iscrowd': obj_info['iscrowd']
                })
        
        return annotations
    
    def convert_split(self, split='train'):
        """转换指定的数据集分割"""
        # 读取有分割标注的图像列表
        seg_split_file = os.path.join(self.voc_root, 'ImageSets', 'Segmentation', f'{split}.txt')
        
        if not os.path.exists(seg_split_file):
            print(f"警告: 未找到分割数据集分割文件 {seg_split_file}")
            print("使用主数据集分割文件...")
            seg_split_file = os.path.join(self.voc_root, 'ImageSets', 'Main', f'{split}.txt')
        
        with open(seg_split_file, 'r') as f:
            image_ids = [line.strip() for line in f.readlines()]
        
        # 初始化COCO格式数据
        coco_data = {
            'info': {
                'description': f'VOC Dataset with SegmentationObject converted to COCO format for {split}',
                'version': '1.0',
                'year': 2024,
                'contributor': 'VOC to COCO Instance Segmentation Converter',
                'date_created': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            },
            'licenses': [],
            'images': [],
            'annotations': [],
            'categories': []
        }
        
        # 添加类别信息
        for cat_name, cat_id in self.categories.items():
            coco_data['categories'].append({
                'id': cat_id,
                'name': cat_name,
                'supercategory': 'object'
            })
        
        annotation_id = 1
        valid_images = 0
        
        for img_id, image_id in enumerate(image_ids, 1):
            # 检查所需文件是否存在
            img_file = os.path.join(self.voc_root, 'JPEGImages', f'{image_id}.jpg')
            xml_file = os.path.join(self.voc_root, 'Annotations', f'{image_id}.xml')
            seg_file = os.path.join(self.voc_root, 'SegmentationObject', f'{image_id}.png')
            
            if not os.path.exists(img_file) or not os.path.exists(xml_file):
                continue
                
            # 获取图像尺寸
            with Image.open(img_file) as img:
                img_width, img_height = img.size
            
            coco_data['images'].append({
                'id': img_id,
                'width': img_width,
                'height': img_height,
                'file_name': f'{image_id}.jpg'
            })
            
            # 解析XML标注
            objects_info, _, _ = self._parse_xml(xml_file)
            
            if len(objects_info) == 0:
                continue
            
            # 提取实例mask
            annotations = self._extract_instance_masks(seg_file, objects_info, img_width, img_height)
            
            for ann in annotations:
                ann['id'] = annotation_id
                ann['image_id'] = img_id
                coco_data['annotations'].append(ann)
                annotation_id += 1
            
            valid_images += 1
            if valid_images % 100 == 0:
                print(f"已处理 {valid_images} 张图像...")
        
        # 保存JSON文件
        os.makedirs(self.output_dir, exist_ok=True)
        output_file = os.path.join(self.output_dir, f'instances_{split}.json')
        with open(output_file, 'w') as f:
            json.dump(coco_data, f, indent=2)
            
        print(f"转换完成: {output_file}")
        print(f"有效图像数量: {valid_images}")
        print(f"总标注数量: {len(coco_data['annotations'])}")
        
        return output_file

def main():
    # 配置路径
    voc_root = 'data/VOCdevkit/VOC2012'  # 修改为你的VOC数据集路径
    output_dir = 'data/voc_coco_format/annotations2012'
    
    converter = VOCToCOCOConverter(voc_root, output_dir)
    
    # 检查SegmentationObject文件夹是否存在
    seg_obj_dir = os.path.join(voc_root, 'SegmentationObject')
    if not os.path.exists(seg_obj_dir):
        print(f"错误: 未找到SegmentationObject文件夹: {seg_obj_dir}")
        print("请确保VOC数据集包含实例分割标注")
        return
    
    # 转换训练集和验证集
    print("开始转换训练集...")
    converter.convert_split('train')
    
    print("\n开始转换验证集...")
    converter.convert_split('val')

if __name__ == '__main__':
    main()

开始转换训练集...
已处理 100 张图像...
已处理 200 张图像...
已处理 300 张图像...
已处理 400 张图像...
已处理 500 张图像...
已处理 600 张图像...
已处理 700 张图像...
已处理 800 张图像...
已处理 900 张图像...
已处理 1000 张图像...
已处理 1100 张图像...
已处理 1200 张图像...
已处理 1300 张图像...
已处理 1400 张图像...
转换完成: data/voc_coco_format/annotations2012/instances_train.json
有效图像数量: 1464
总标注数量: 3508

开始转换验证集...
已处理 100 张图像...
已处理 200 张图像...
已处理 300 张图像...
已处理 400 张图像...
已处理 500 张图像...
已处理 600 张图像...
已处理 700 张图像...
已处理 800 张图像...
已处理 900 张图像...
已处理 1000 张图像...
已处理 1100 张图像...
已处理 1200 张图像...
已处理 1300 张图像...
已处理 1400 张图像...
转换完成: data/voc_coco_format/annotations2012/instances_val.json
有效图像数量: 1449
总标注数量: 3426


In [5]:
import os
import json
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
from pycocotools import mask as maskUtils
from datetime import datetime
from collections import defaultdict

class VOCToCOCOConverter:
    def __init__(self, voc_root, output_dir):
        self.voc_root = voc_root
        self.output_dir = output_dir
        self.categories = self._get_voc_categories()
        
    def _get_voc_categories(self):
        """VOC 2012的20个类别"""
        categories = [
            'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
            'bus', 'car', 'cat', 'chair', 'cow',
            'diningtable', 'dog', 'horse', 'motorbike', 'person',
            'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
        ]
        return {cat: i+1 for i, cat in enumerate(categories)}
    
    def _parse_xml(self, xml_file):
        """解析VOC XML标注文件"""
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        objects_info = []
        size = root.find('size')
        img_width = int(size.find('width').text)
        img_height = int(size.find('height').text)
        
        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in self.categories:
                continue
                
            bbox_elem = obj.find('bndbox')
            xmin = float(bbox_elem.find('xmin').text)
            ymin = float(bbox_elem.find('ymin').text)
            xmax = float(bbox_elem.find('xmax').text)
            ymax = float(bbox_elem.find('ymax').text)
            
            # COCO格式: [x, y, width, height]
            bbox = [xmin, ymin, xmax - xmin, ymax - ymin]
            
            objects_info.append({
                'category_name': name,
                'category_id': self.categories[name],
                'bbox': bbox,
                'iscrowd': 0
            })
            
        return objects_info, img_width, img_height
    
    def _analyze_segmentation_image(self, seg_file):
        """分析分割图像中的像素值分布"""
        if not os.path.exists(seg_file):
            return None, None
            
        seg_img = np.array(Image.open(seg_file))
        
        # 获取所有非背景和非边界的像素值
        unique_values = np.unique(seg_img)
        # 过滤掉背景(0)和边界(255)
        object_values = unique_values[(unique_values != 0) & (unique_values != 255)]
        
        return seg_img, object_values
    
    def _calculate_overlap(self, bbox, mask, threshold=0.3):
        """计算边界框与mask的重叠程度"""
        x, y, w, h = [int(v) for v in bbox]
        
        # 确保边界框在图像范围内
        img_h, img_w = mask.shape
        x = max(0, min(x, img_w-1))
        y = max(0, min(y, img_h-1))
        w = min(w, img_w - x)
        h = min(h, img_h - y)
        
        if w <= 0 or h <= 0:
            return 0.0
        
        # 计算边界框内的mask像素数
        bbox_mask_pixels = np.sum(mask[y:y+h, x:x+w])
        bbox_total_pixels = w * h
        
        # 计算重叠比例
        overlap_ratio = bbox_mask_pixels / bbox_total_pixels if bbox_total_pixels > 0 else 0
        
        return overlap_ratio
    
    def _find_best_mask_for_bbox(self, bbox, seg_img, object_values, used_masks, min_overlap=0.1):
        """为边界框找到最佳匹配的mask"""
        best_mask = None
        best_overlap = 0
        best_value = None
        
        for val in object_values:
            if val in used_masks:
                continue
                
            # 创建当前像素值的mask
            current_mask = (seg_img == val).astype(np.uint8)
            
            # 计算重叠度
            overlap = self._calculate_overlap(bbox, current_mask, min_overlap)
            
            if overlap > best_overlap and overlap >= min_overlap:
                best_overlap = overlap
                best_mask = current_mask
                best_value = val
        
        return best_mask, best_value, best_overlap
    
    def _create_bbox_mask(self, bbox, img_width, img_height):
        """从边界框创建简单的矩形mask作为后备方案"""
        x, y, w, h = [int(v) for v in bbox]
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        
        # 确保坐标在有效范围内
        x = max(0, min(x, img_width-1))
        y = max(0, min(y, img_height-1))
        w = min(w, img_width - x)
        h = min(h, img_height - y)
        
        if w > 0 and h > 0:
            mask[y:y+h, x:x+w] = 1
            
        return mask
    
    def _extract_instance_masks_improved(self, seg_file, objects_info, img_width, img_height):
        """改进的实例mask提取算法"""
        if not os.path.exists(seg_file):
            # 如果没有分割文件，使用边界框创建mask
            print(f"警告: 分割文件不存在 {os.path.basename(seg_file)}, 使用边界框mask")
            annotations = []
            for obj_info in objects_info:
                mask = self._create_bbox_mask(obj_info['bbox'], img_width, img_height)
                area = float(np.sum(mask))
                
                if area > 0:
                    rle = maskUtils.encode(np.asfortranarray(mask))
                    rle['counts'] = rle['counts'].decode('utf-8')
                    
                    annotations.append({
                        'category_id': obj_info['category_id'],
                        'bbox': obj_info['bbox'],
                        'area': area,
                        'segmentation': rle,
                        'iscrowd': obj_info['iscrowd']
                    })
            return annotations
            
        # 分析分割图像
        seg_img, object_values = self._analyze_segmentation_image(seg_file)
        
        if object_values is None or len(object_values) == 0:
            print(f"警告: 分割图像中没有有效对象 {os.path.basename(seg_file)}")
            return []
        
        annotations = []
        used_masks = set()
        unmatched_objects = []
        
        # 第一轮：为每个边界框寻找最佳匹配的mask
        for obj_info in objects_info:
            best_mask, best_value, overlap = self._find_best_mask_for_bbox(
                obj_info['bbox'], seg_img, object_values, used_masks, min_overlap=0.1
            )
            
            if best_mask is not None and overlap > 0.1:
                # 找到了合适的mask
                area = float(np.sum(best_mask))
                
                if area > 0:
                    # 标记该mask已被使用
                    used_masks.add(best_value)
                    
                    # 从mask重新计算精确的边界框
                    coords = np.where(best_mask)
                    if len(coords[0]) > 0:
                        y_min, y_max = coords[0].min(), coords[0].max()
                        x_min, x_max = coords[1].min(), coords[1].max()
                        precise_bbox = [float(x_min), float(y_min), 
                                      float(x_max - x_min + 1), float(y_max - y_min + 1)]
                    else:
                        precise_bbox = obj_info['bbox']
                    
                    # 转换为RLE格式
                    rle = maskUtils.encode(np.asfortranarray(best_mask))
                    rle['counts'] = rle['counts'].decode('utf-8')
                    
                    annotations.append({
                        'category_id': obj_info['category_id'],
                        'bbox': precise_bbox,
                        'area': area,
                        'segmentation': rle,
                        'iscrowd': obj_info['iscrowd']
                    })
                else:
                    unmatched_objects.append(obj_info)
            else:
                unmatched_objects.append(obj_info)
        
        # 第二轮：为未匹配的对象降低匹配阈值或使用边界框mask
        for obj_info in unmatched_objects:
            # 尝试更低的重叠阈值
            best_mask, best_value, overlap = self._find_best_mask_for_bbox(
                obj_info['bbox'], seg_img, object_values, used_masks, min_overlap=0.05
            )
            
            if best_mask is not None and overlap > 0.05:
                area = float(np.sum(best_mask))
                if area > 0:
                    used_masks.add(best_value)
                    
                    coords = np.where(best_mask)
                    if len(coords[0]) > 0:
                        y_min, y_max = coords[0].min(), coords[0].max()
                        x_min, x_max = coords[1].min(), coords[1].max()
                        precise_bbox = [float(x_min), float(y_min), 
                                      float(x_max - x_min + 1), float(y_max - y_min + 1)]
                    else:
                        precise_bbox = obj_info['bbox']
                    
                    rle = maskUtils.encode(np.asfortranarray(best_mask))
                    rle['counts'] = rle['counts'].decode('utf-8')
                    
                    annotations.append({
                        'category_id': obj_info['category_id'],
                        'bbox': precise_bbox,
                        'area': area,
                        'segmentation': rle,
                        'iscrowd': obj_info['iscrowd']
                    })
                    continue
            
            # 最后的备选方案：使用边界框mask
            bbox_mask = self._create_bbox_mask(obj_info['bbox'], img_width, img_height)
            area = float(np.sum(bbox_mask))
            
            if area > 0:
                rle = maskUtils.encode(np.asfortranarray(bbox_mask))
                rle['counts'] = rle['counts'].decode('utf-8')
                
                annotations.append({
                    'category_id': obj_info['category_id'],
                    'bbox': obj_info['bbox'],
                    'area': area,
                    'segmentation': rle,
                    'iscrowd': obj_info['iscrowd']
                })
        
        # 第三轮：处理未被任何边界框匹配的分割区域
        unused_values = set(object_values) - used_masks
        for val in unused_values:
            mask = (seg_img == val).astype(np.uint8)
            area = float(np.sum(mask))
            
            if area > 100:  # 只处理足够大的区域
                # 尝试根据mask的位置推断类别
                coords = np.where(mask)
                if len(coords[0]) > 0:
                    y_center = (coords[0].min() + coords[0].max()) / 2
                    x_center = (coords[1].min() + coords[1].max()) / 2
                    
                    # 寻找最近的边界框来推断类别
                    min_dist = float('inf')
                    closest_category = 1  # 默认类别
                    
                    for obj_info in objects_info:
                        bbox_x, bbox_y, bbox_w, bbox_h = obj_info['bbox']
                        bbox_center_x = bbox_x + bbox_w / 2
                        bbox_center_y = bbox_y + bbox_h / 2
                        
                        dist = ((x_center - bbox_center_x) ** 2 + (y_center - bbox_center_y) ** 2) ** 0.5
                        if dist < min_dist:
                            min_dist = dist
                            closest_category = obj_info['category_id']
                    
                    # 计算精确边界框
                    y_min, y_max = coords[0].min(), coords[0].max()
                    x_min, x_max = coords[1].min(), coords[1].max()
                    bbox = [float(x_min), float(y_min), 
                           float(x_max - x_min + 1), float(y_max - y_min + 1)]
                    
                    rle = maskUtils.encode(np.asfortranarray(mask))
                    rle['counts'] = rle['counts'].decode('utf-8')
                    
                    annotations.append({
                        'category_id': closest_category,
                        'bbox': bbox,
                        'area': area,
                        'segmentation': rle,
                        'iscrowd': 0
                    })
        
        return annotations
    
    def convert_split(self, split='train'):
        """转换指定的数据集分割"""
        # 读取有分割标注的图像列表
        seg_split_file = os.path.join(self.voc_root, 'ImageSets', 'Segmentation', f'{split}.txt')
        
        if not os.path.exists(seg_split_file):
            print(f"警告: 未找到分割数据集分割文件 {seg_split_file}")
            print("使用主数据集分割文件...")
            seg_split_file = os.path.join(self.voc_root, 'ImageSets', 'Main', f'{split}.txt')
        
        with open(seg_split_file, 'r') as f:
            image_ids = [line.strip() for line in f.readlines()]
        
        # 初始化COCO格式数据
        coco_data = {
            'info': {
                'description': f'VOC Dataset with SegmentationObject converted to COCO format for {split}',
                'version': '2.0',
                'year': 2024,
                'contributor': 'Improved VOC to COCO Instance Segmentation Converter',
                'date_created': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            },
            'licenses': [],
            'images': [],
            'annotations': [],
            'categories': []
        }
        
        # 添加类别信息
        for cat_name, cat_id in self.categories.items():
            coco_data['categories'].append({
                'id': cat_id,
                'name': cat_name,
                'supercategory': 'object'
            })
        
        annotation_id = 1
        valid_images = 0
        total_processed = 0
        skipped_no_objects = 0
        skipped_no_files = 0
        
        print(f"开始处理 {len(image_ids)} 张图像...")
        
        for img_id, image_id in enumerate(image_ids, 1):
            total_processed += 1
            
            # 检查所需文件是否存在
            img_file = os.path.join(self.voc_root, 'JPEGImages', f'{image_id}.jpg')
            xml_file = os.path.join(self.voc_root, 'Annotations', f'{image_id}.xml')
            seg_file = os.path.join(self.voc_root, 'SegmentationObject', f'{image_id}.png')
            
            if not os.path.exists(img_file) or not os.path.exists(xml_file):
                skipped_no_files += 1
                continue
                
            # 获取图像尺寸
            try:
                with Image.open(img_file) as img:
                    img_width, img_height = img.size
            except Exception as e:
                print(f"无法读取图像 {image_id}: {e}")
                continue
            
            coco_data['images'].append({
                'id': img_id,
                'width': img_width,
                'height': img_height,
                'file_name': f'{image_id}.jpg'
            })
            
            # 解析XML标注
            try:
                objects_info, _, _ = self._parse_xml(xml_file)
            except Exception as e:
                print(f"解析XML失败 {image_id}: {e}")
                continue
            
            if len(objects_info) == 0:
                skipped_no_objects += 1
                continue
            
            # 提取实例mask（改进的算法）
            annotations = self._extract_instance_masks_improved(seg_file, objects_info, img_width, img_height)
            
            if len(annotations) > 0:
                for ann in annotations:
                    ann['id'] = annotation_id
                    ann['image_id'] = img_id
                    coco_data['annotations'].append(ann)
                    annotation_id += 1
                
                valid_images += 1
            
            if total_processed % 100 == 0:
                print(f"已处理 {total_processed} 张图像, 有效: {valid_images}")
        
        # 保存JSON文件
        os.makedirs(self.output_dir, exist_ok=True)
        output_file = os.path.join(self.output_dir, f'instances_{split}.json')
        with open(output_file, 'w') as f:
            json.dump(coco_data, f, indent=2)
            
        print(f"\n转换完成: {output_file}")
        print(f"总处理图像: {total_processed}")
        print(f"有效图像数量: {valid_images}")
        print(f"跳过（无文件）: {skipped_no_files}")
        print(f"跳过（无对象）: {skipped_no_objects}")
        print(f"总标注数量: {len(coco_data['annotations'])}")
        print(f"有效率: {valid_images/total_processed*100:.1f}%")
        
        return output_file

def main():
    # 配置路径
    voc_root = 'data/VOCdevkit/VOC2012'  # 修改为你的VOC数据集路径
    output_dir = 'data/voc_coco_format/annotations2012'
    
    converter = VOCToCOCOConverter(voc_root, output_dir)
    
    # 检查必要文件夹
    required_dirs = ['JPEGImages', 'Annotations', 'SegmentationObject', 'ImageSets']
    for dir_name in required_dirs:
        dir_path = os.path.join(voc_root, dir_name)
        if not os.path.exists(dir_path):
            print(f"错误: 未找到必需文件夹: {dir_path}")
            return
    
    # 检查分割文件数量
    seg_obj_dir = os.path.join(voc_root, 'SegmentationObject')
    seg_files = [f for f in os.listdir(seg_obj_dir) if f.endswith('.png')]
    print(f"SegmentationObject中有 {len(seg_files)} 个分割文件")
    
    # 转换训练集和验证集
    print("开始转换训练集...")
    converter.convert_split('train')
    
    print("\n开始转换验证集...")
    converter.convert_split('val')

if __name__ == '__main__':
    main()

SegmentationObject中有 2913 个分割文件
开始转换训练集...
开始处理 1464 张图像...
已处理 100 张图像, 有效: 100
已处理 200 张图像, 有效: 200
已处理 300 张图像, 有效: 300
已处理 400 张图像, 有效: 400
已处理 500 张图像, 有效: 500
已处理 600 张图像, 有效: 600
已处理 700 张图像, 有效: 700
已处理 800 张图像, 有效: 800
已处理 900 张图像, 有效: 900
已处理 1000 张图像, 有效: 1000
已处理 1100 张图像, 有效: 1100
已处理 1200 张图像, 有效: 1200
已处理 1300 张图像, 有效: 1300
已处理 1400 张图像, 有效: 1400

转换完成: data/voc_coco_format/annotations2012/instances_train.json
总处理图像: 1464
有效图像数量: 1464
跳过（无文件）: 0
跳过（无对象）: 0
总标注数量: 3567
有效率: 100.0%

开始转换验证集...
开始处理 1449 张图像...
已处理 100 张图像, 有效: 100
已处理 200 张图像, 有效: 200
已处理 300 张图像, 有效: 300
已处理 400 张图像, 有效: 400
已处理 500 张图像, 有效: 500
已处理 600 张图像, 有效: 600
已处理 700 张图像, 有效: 700
已处理 800 张图像, 有效: 800
已处理 900 张图像, 有效: 900
已处理 1000 张图像, 有效: 1000
已处理 1100 张图像, 有效: 1100
已处理 1200 张图像, 有效: 1200
已处理 1300 张图像, 有效: 1300
已处理 1400 张图像, 有效: 1400

转换完成: data/voc_coco_format/annotations2012/instances_val.json
总处理图像: 1449
有效图像数量: 1449
跳过（无文件）: 0
跳过（无对象）: 0
总标注数量: 3489
有效率: 100.0%


In [8]:
import os
import json
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
from pycocotools import mask as maskUtils
from datetime import datetime
from collections import defaultdict
import random

class VOCToCOCOConverter:
    def __init__(self, voc_root, output_dir):
        self.voc_root = voc_root
        self.output_dir = output_dir
        self.categories = self._get_voc_categories()
        
    def _get_voc_categories(self):
        """VOC 2012的20个类别"""
        categories = [
            'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
            'bus', 'car', 'cat', 'chair', 'cow',
            'diningtable', 'dog', 'horse', 'motorbike', 'person',
            'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
        ]
        return {cat: i+1 for i, cat in enumerate(categories)}
    
    def _parse_xml(self, xml_file):
        """解析VOC XML标注文件"""
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        objects_info = []
        size = root.find('size')
        img_width = int(size.find('width').text)
        img_height = int(size.find('height').text)
        
        for obj in root.findall('object'):
            name = obj.find('name').text
            if name not in self.categories:
                continue
                
            bbox_elem = obj.find('bndbox')
            xmin = float(bbox_elem.find('xmin').text)
            ymin = float(bbox_elem.find('ymin').text)
            xmax = float(bbox_elem.find('xmax').text)
            ymax = float(bbox_elem.find('ymax').text)
            
            # COCO格式: [x, y, width, height]
            bbox = [xmin, ymin, xmax - xmin, ymax - ymin]
            
            objects_info.append({
                'category_name': name,
                'category_id': self.categories[name],
                'bbox': bbox,
                'iscrowd': 0
            })
            
        return objects_info, img_width, img_height
    
    def _extract_instance_masks(self, seg_file, objects_info, img_width, img_height):
        """从VOC分割图像中提取每个实例的mask"""
        if not os.path.exists(seg_file):
            return []
            
        # 读取分割图像
        seg_img = np.array(Image.open(seg_file))
        
        # 获取所有唯一的像素值（除了背景0和边界255）
        unique_values = np.unique(seg_img)
        unique_values = unique_values[(unique_values != 0) & (unique_values != 255)]
        
        annotations = []
        
        # 为每个检测到的对象匹配分割mask
        for obj_info in objects_info:
            # 在分割图像中寻找与边界框重叠最大的分割区域
            x, y, w, h = obj_info['bbox']
            x, y, w, h = int(x), int(y), int(w), int(h)
            
            # 获取边界框区域内的分割值
            bbox_region = seg_img[y:y+h, x:x+w]
            bbox_unique = np.unique(bbox_region)
            bbox_unique = bbox_unique[(bbox_unique != 0) & (bbox_unique != 255)]
            
            if len(bbox_unique) == 0:
                # 如果没有找到分割区域，使用边界框创建矩形mask
                mask = np.zeros((img_height, img_width), dtype=np.uint8)
                mask[y:y+h, x:x+w] = 1
            else:
                # 选择在边界框内占比最大的分割值
                best_value = None
                max_overlap = 0
                
                for val in bbox_unique:
                    if val in unique_values:
                        val_mask = (seg_img == val).astype(np.uint8)
                        overlap = np.sum(val_mask[y:y+h, x:x+w])
                        if overlap > max_overlap:
                            max_overlap = overlap
                            best_value = val
                
                if best_value is not None:
                    mask = (seg_img == best_value).astype(np.uint8)
                    # 从unique_values中移除已使用的值，避免重复使用
                    unique_values = unique_values[unique_values != best_value]
                else:
                    # 备选方案：使用边界框
                    mask = np.zeros((img_height, img_width), dtype=np.uint8)
                    mask[y:y+h, x:x+w] = 1
            
            # 计算面积
            area = float(np.sum(mask))
            
            if area > 0:
                # 转换为RLE格式
                rle = maskUtils.encode(np.asfortranarray(mask))
                rle['counts'] = rle['counts'].decode('utf-8')
                
                # 从mask重新计算更精确的边界框
                coords = np.where(mask)
                if len(coords[0]) > 0:
                    y_min, y_max = coords[0].min(), coords[0].max()
                    x_min, x_max = coords[1].min(), coords[1].max()
                    precise_bbox = [float(x_min), float(y_min), 
                                  float(x_max - x_min + 1), float(y_max - y_min + 1)]
                else:
                    precise_bbox = obj_info['bbox']
                
                annotations.append({
                    'category_id': obj_info['category_id'],
                    'bbox': precise_bbox,
                    'area': area,
                    'segmentation': rle,
                    'iscrowd': obj_info['iscrowd']
                })
        
        return annotations
    
    def get_all_segmentation_images(self):
        """获取所有有分割标注的图像ID"""
        # 优先使用分割数据集的trainval文件
        seg_trainval_file = os.path.join(self.voc_root, 'ImageSets', 'Segmentation', 'trainval.txt')
        
        if os.path.exists(seg_trainval_file):
            print("使用分割数据集的trainval.txt")
            with open(seg_trainval_file, 'r') as f:
                all_image_ids = [line.strip() for line in f.readlines()]
        else:
            # 如果没有trainval.txt，尝试合并train.txt和val.txt
            train_file = os.path.join(self.voc_root, 'ImageSets', 'Segmentation', 'train.txt')
            val_file = os.path.join(self.voc_root, 'ImageSets', 'Segmentation', 'val.txt')
            
            all_image_ids = []
            if os.path.exists(train_file):
                with open(train_file, 'r') as f:
                    all_image_ids.extend([line.strip() for line in f.readlines()])
            
            if os.path.exists(val_file):
                with open(val_file, 'r') as f:
                    all_image_ids.extend([line.strip() for line in f.readlines()])
            
            if not all_image_ids:
                # 最后的备选方案：从SegmentationObject目录直接获取
                seg_obj_dir = os.path.join(self.voc_root, 'SegmentationObject')
                if os.path.exists(seg_obj_dir):
                    print("从SegmentationObject目录获取图像列表")
                    seg_files = [f for f in os.listdir(seg_obj_dir) if f.endswith('.png')]
                    all_image_ids = [f[:-4] for f in seg_files]  # 移除.png后缀
                else:
                    raise ValueError("无法找到分割数据集的图像列表")
        
        # 去重并排序
        all_image_ids = sorted(list(set(all_image_ids)))
        print(f"找到 {len(all_image_ids)} 个有分割标注的图像")
        
        return all_image_ids
    
    def split_train_val(self, all_image_ids, train_ratio=0.8):
        """按比例划分训练集和验证集"""
        # 设置随机种子保证结果可复现
        random.seed(42)
        
        # 打乱图像ID列表
        shuffled_ids = all_image_ids.copy()
        random.shuffle(shuffled_ids)
        
        # 计算分割点
        total_count = len(shuffled_ids)
        train_count = int(total_count * train_ratio)
        
        train_ids = shuffled_ids[:train_count]
        val_ids = shuffled_ids[train_count:]
        
        print(f"数据集划分:")
        print(f"  训练集: {len(train_ids)} 个图像 ({len(train_ids)/total_count*100:.1f}%)")
        print(f"  验证集: {len(val_ids)} 个图像 ({len(val_ids)/total_count*100:.1f}%)")
        
        return train_ids, val_ids

    def convert_split(self, image_ids, split='train'):
        """转换指定的数据集分割"""
        
        # 初始化COCO格式数据
        coco_data = {
            'info': {
                'description': f'VOC Dataset with SegmentationObject converted to COCO format for {split}',
                'version': '1.0',
                'year': 2024,
                'contributor': 'VOC to COCO Instance Segmentation Converter',
                'date_created': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            },
            'licenses': [],
            'images': [],
            'annotations': [],
            'categories': []
        }
        
        # 添加类别信息
        for cat_name, cat_id in self.categories.items():
            coco_data['categories'].append({
                'id': cat_id,
                'name': cat_name,
                'supercategory': 'object'
            })
        
        annotation_id = 1
        valid_images = 0
        
        for img_id, image_id in enumerate(image_ids, 1):
            # 检查所需文件是否存在
            img_file = os.path.join(self.voc_root, 'JPEGImages', f'{image_id}.jpg')
            xml_file = os.path.join(self.voc_root, 'Annotations', f'{image_id}.xml')
            seg_file = os.path.join(self.voc_root, 'SegmentationObject', f'{image_id}.png')
            
            if not os.path.exists(img_file) or not os.path.exists(xml_file):
                continue
                
            # 获取图像尺寸
            with Image.open(img_file) as img:
                img_width, img_height = img.size
            
            coco_data['images'].append({
                'id': img_id,
                'width': img_width,
                'height': img_height,
                'file_name': f'{image_id}.jpg'
            })
            
            # 解析XML标注
            objects_info, _, _ = self._parse_xml(xml_file)
            
            if len(objects_info) == 0:
                continue
            
            # 提取实例mask
            annotations = self._extract_instance_masks(seg_file, objects_info, img_width, img_height)
            
            for ann in annotations:
                ann['id'] = annotation_id
                ann['image_id'] = img_id
                coco_data['annotations'].append(ann)
                annotation_id += 1
            
            valid_images += 1
            if valid_images % 100 == 0:
                print(f"已处理 {valid_images} 张图像...")
        
        # 保存JSON文件
        os.makedirs(self.output_dir, exist_ok=True)
        output_file = os.path.join(self.output_dir, f'instances_{split}.json')
        with open(output_file, 'w') as f:
            json.dump(coco_data, f, indent=2)
            
        print(f"转换完成: {output_file}")
        print(f"有效图像数量: {valid_images}")
        print(f"总标注数量: {len(coco_data['annotations'])}")
        
        return output_file

    def convert_all_data(self, train_ratio=0.8):
        """转换所有数据并按比例划分训练集和验证集"""
        print("开始数据转换...")
        
        # 1. 获取所有有分割标注的图像
        all_image_ids = self.get_all_segmentation_images()
        
        # 2. 按比例划分训练集和验证集
        train_ids, val_ids = self.split_train_val(all_image_ids, train_ratio)
        
        # 3. 转换训练集
        print(f"\n转换训练集 ({len(train_ids)} 个图像)...")
        train_file = self.convert_split(train_ids, 'train')
        
        # 4. 转换验证集
        print(f"\n转换验证集 ({len(val_ids)} 个图像)...")
        val_file = self.convert_split(val_ids, 'val')
        
        return train_file, val_file

def main():
    # 配置路径
    voc_root = 'data/VOCdevkit/VOC2012'  # 修改为你的VOC数据集路径
    output_dir = 'data/voc_coco_format2/annotations'
    
    converter = VOCToCOCOConverter(voc_root, output_dir)
    
    # 检查SegmentationObject文件夹是否存在
    seg_obj_dir = os.path.join(voc_root, 'SegmentationObject')
    if not os.path.exists(seg_obj_dir):
        print(f"错误: 未找到SegmentationObject文件夹: {seg_obj_dir}")
        print("请确保VOC数据集包含实例分割标注")
        return
    
    # 使用4:1比例划分训练集和验证集
    print("=" * 60)
    print("VOC数据集转换为COCO格式 (训练:验证 = 4:1)")
    print("=" * 60)
    
    try:
        train_file, val_file = converter.convert_all_data(train_ratio=0.8)
        
        print("\n" + "=" * 60)
        print("✓ 数据转换完成!")
        print(f"✓ 训练集标注: {train_file}")
        print(f"✓ 验证集标注: {val_file}")
        print("=" * 60)
        
    except Exception as e:
        print(f"❌ 转换失败: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

VOC数据集转换为COCO格式 (训练:验证 = 4:1)
开始数据转换...
使用分割数据集的trainval.txt
找到 2913 个有分割标注的图像
数据集划分:
  训练集: 2330 个图像 (80.0%)
  验证集: 583 个图像 (20.0%)

转换训练集 (2330 个图像)...
已处理 100 张图像...
已处理 200 张图像...
已处理 300 张图像...
已处理 400 张图像...
已处理 500 张图像...
已处理 600 张图像...
已处理 700 张图像...
已处理 800 张图像...
已处理 900 张图像...
已处理 1000 张图像...
已处理 1100 张图像...
已处理 1200 张图像...
已处理 1300 张图像...
已处理 1400 张图像...
已处理 1500 张图像...
已处理 1600 张图像...
已处理 1700 张图像...
已处理 1800 张图像...
已处理 1900 张图像...
已处理 2000 张图像...
已处理 2100 张图像...
已处理 2200 张图像...
已处理 2300 张图像...
转换完成: data/voc_coco_format2/annotations/instances_train.json
有效图像数量: 2330
总标注数量: 5558

转换验证集 (583 个图像)...
已处理 100 张图像...
已处理 200 张图像...
已处理 300 张图像...
已处理 400 张图像...
已处理 500 张图像...
转换完成: data/voc_coco_format2/annotations/instances_val.json
有效图像数量: 583
总标注数量: 1376

✓ 数据转换完成!
✓ 训练集标注: data/voc_coco_format2/annotations/instances_train.json
✓ 验证集标注: data/voc_coco_format2/annotations/instances_val.json
